# Workshop Setup: Accelerated Discovery Framework

This notebook automatically sets up the complete environment for the Accelerated Discovery Framework workshop.

**Prerequisites:**
- Python 3.8+ installed
- GitHub access (for cloning)
- Internet connection

**What this notebook does:**
1. Installs Docker (if needed)
2. Clones the repository to a separate workshop directory (`~/akd-workshop/`)
3. Sets up Python environment with UV
4. Runs the standalone setup script
5. Verifies everything works by running the deep search testing notebook

**Important:** 
- This setup creates a separate workspace in `~/akd-workshop/` to avoid affecting your current environment
- Run cells in order and wait for each to complete before proceeding
- The setup is completely isolated from your current working directory

## Step 1: System Detection and Docker Installation

In [ ]:
import subprocess
import sys
import os
import platform
import time
from pathlib import Path

def run_command(cmd, check=True, shell=False):
    """Run a command and return the result"""
    try:
        if shell:
            result = subprocess.run(cmd, shell=True, capture_output=True, text=True, check=check)
        else:
            result = subprocess.run(cmd, capture_output=True, text=True, check=check)
        return result
    except subprocess.CalledProcessError as e:
        print(f"❌ Command failed: {' '.join(cmd) if isinstance(cmd, list) else cmd}")
        print(f"Error: {e.stderr}")
        if check:
            raise
        return e

def check_command_exists(cmd):
    """Check if a command exists on the system"""
    try:
        subprocess.run(["which", cmd], check=True, capture_output=True)
        return True
    except subprocess.CalledProcessError:
        return False

# Detect operating system
system = platform.system().lower()
print(f"🖥️  Detected OS: {system}")
print(f"🐍 Python version: {sys.version}")

### Docker Installation Check and Setup

In [ ]:
# Check if Docker is installed
docker_installed = check_command_exists("docker")

if docker_installed:
    print("✅ Docker is already installed")
    # Check if Docker daemon is running
    try:
        result = run_command(["docker", "info"], check=False)
        if result.returncode == 0:
            print("✅ Docker daemon is running")
        else:
            print("⚠️  Docker is installed but daemon is not running")
            print("Please start Docker Desktop or Docker service manually")
            print("On macOS/Windows: Start Docker Desktop application")
            print("On Linux: Run 'sudo systemctl start docker'")
    except Exception as e:
        print(f"⚠️  Could not check Docker status: {e}")
else:
    print("❌ Docker not found. Installing Docker...")
    
    if system == "darwin":  # macOS
        print("🍺 Installing Docker via Homebrew...")
        # Check if Homebrew is installed
        if not check_command_exists("brew"):
            print("Installing Homebrew first...")
            install_brew = '/bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"'
            run_command(install_brew, shell=True)
        
        # Install Docker Desktop
        run_command(["brew", "install", "--cask", "docker"])
        print("📱 Please manually start Docker Desktop from Applications folder")
        print("Wait for Docker to start, then continue with the next cell")
        
    elif system == "linux":
        print("🐧 Installing Docker on Linux...")
        # Install Docker using the official script
        install_script = "curl -fsSL https://get.docker.com -o get-docker.sh && sh get-docker.sh"
        run_command(install_script, shell=True)
        
        # Start Docker service
        run_command(["sudo", "systemctl", "start", "docker"])
        run_command(["sudo", "systemctl", "enable", "docker"])
        
        # Add user to docker group
        run_command(["sudo", "usermod", "-aG", "docker", os.getenv("USER")])
        print("✅ Docker installed. You may need to log out and back in for group changes to take effect.")
        
    elif system == "windows":
        print("🪟 For Windows, please manually download and install Docker Desktop from:")
        print("https://desktop.docker.com/win/main/amd64/Docker%20Desktop%20Installer.exe")
        print("After installation, start Docker Desktop and continue with the next cell.")
    
    else:
        print(f"❌ Unsupported operating system: {system}")
        print("Please install Docker manually from https://docs.docker.com/get-docker/")

### Verify Docker Installation

In [ ]:
# Wait for Docker to be ready
print("🔄 Waiting for Docker to be ready...")

max_attempts = 30
for attempt in range(max_attempts):
    try:
        result = run_command(["docker", "info"], check=False)
        if result.returncode == 0:
            print("✅ Docker is ready!")
            break
    except Exception:
        pass
    
    if attempt < max_attempts - 1:
        print(f"⏳ Attempt {attempt + 1}/{max_attempts} - Docker not ready yet, waiting...")
        time.sleep(5)
    else:
        print("❌ Docker failed to start after 2.5 minutes")
        print("Please ensure Docker is installed and running, then restart this notebook")
        raise RuntimeError("Docker not available")

# Test Docker with hello-world
print("🧪 Testing Docker with hello-world container...")
result = run_command(["docker", "run", "--rm", "hello-world"])
if "Hello from Docker!" in result.stdout:
    print("✅ Docker test successful!")
else:
    print("⚠️  Docker test output unexpected, but continuing...")

## Step 2: Repository Setup

In [ ]:
# Set up workspace (separate from your current working directory)
workspace_dir = Path.home() / "akd-workshop" / "accelerated-discovery"
workspace_dir.parent.mkdir(exist_ok=True)

print(f"📁 Setting up workshop workspace at: {workspace_dir}")
print("   (This is separate from your current development environment)")

# Clone repository if it doesn't exist
if workspace_dir.exists():
    print("✅ Repository directory already exists")
    os.chdir(workspace_dir)
    
    # Check if it's a git repo and pull latest changes
    if (workspace_dir / ".git").exists():
        print("🔄 Pulling latest changes...")
        run_command(["git", "pull"])
    else:
        print("⚠️  Directory exists but is not a git repository")
        print("Please remove the directory or clone manually")
else:
    print("📥 Cloning repository...")
    # Change to parent directory for cloning
    os.chdir(workspace_dir.parent)
    run_command(["git", "clone", "https://github.com/mramasub/accelerated-discovery.git"])
    os.chdir(workspace_dir)
    print("✅ Repository cloned successfully")

print(f"📍 Current directory: {os.getcwd()}")
print(f"📂 Contents: {list(os.listdir('.'))}")

## Step 3: Python Environment Setup

In [ ]:
# Install UV if not present
if not check_command_exists("uv"):
    print("📦 Installing UV (Python package manager)...")
    install_uv = "curl -LsSf https://astral.sh/uv/install.sh | sh"
    run_command(install_uv, shell=True)
    
    # Add to PATH for current session
    home = Path.home()
    uv_path = home / ".local" / "bin"
    if str(uv_path) not in os.environ["PATH"]:
        os.environ["PATH"] = f"{uv_path}:{os.environ['PATH']}"
    
    print("✅ UV installed")
else:
    print("✅ UV already available")

# Check UV version
result = run_command(["uv", "--version"])
print(f"📦 UV version: {result.stdout.strip()}")

In [ ]:
# Create virtual environment and install dependencies
print("🐍 Setting up Python virtual environment...")

# Create virtual environment with Python 3.12
run_command(["uv", "venv", "--python", "3.12"])
print("✅ Virtual environment created")

# Install dependencies using UV
print("📦 Installing dependencies...")
run_command(["uv", "sync"])
print("✅ Dependencies installed")

# Verify installation
print("🔍 Verifying package installation...")
result = run_command(["uv", "run", "python", "-c", "import akd; print('AKD framework imported successfully')"])
if "successfully" in result.stdout:
    print("✅ AKD framework installed correctly")
else:
    print("⚠️  There might be an issue with the installation")
    print(result.stdout)
    print(result.stderr)

## Step 4: Environment Variables Setup

In [ ]:
# Check for .env file
env_file = Path(".env")
env_example = Path(".env.example")

if not env_file.exists():
    if env_example.exists():
        print("📄 Copying .env.example to .env")
        import shutil
        shutil.copy(env_example, env_file)
        print("✅ .env file created")
    else:
        print("📄 Creating basic .env file")
        with open(env_file, "w") as f:
            f.write("# Environment variables for AKD Framework\n")
            f.write("# Add your API keys here:\n")
            f.write("# OPENAI_API_KEY=your_openai_key_here\n")
            f.write("# ANTHROPIC_API_KEY=your_anthropic_key_here\n")
else:
    print("✅ .env file already exists")

print("⚠️  IMPORTANT: You'll need to add your API keys to the .env file for full functionality")
print(f"📝 Edit {env_file.absolute()} and add your API keys")
print("For this workshop, you can continue without API keys for basic functionality")

## Step 5: Run Standalone Setup Script

In [ ]:
# Run the setup_standalone.sh script
print("🚀 Running standalone setup script...")
print("This will start SearxNG Docker container for search functionality")
print("⚠️  Note: This may conflict if you already have SearxNG running on port 8080")

try:
    result = run_command(["bash", "scripts/setup_standalone.sh"])
    print("✅ Standalone setup completed successfully!")
    print("📡 SearxNG search service is running at http://localhost:8080")
except subprocess.CalledProcessError as e:
    print("❌ Setup script failed")
    print("Error output:")
    print(e.stderr)
    print("\nTrying to diagnose the issue...")
    
    # Check if docker-compose files exist
    compose_file = Path("config/searxng/docker-compose.yml")
    settings_file = Path("config/searxng/settings.yml")
    
    if not compose_file.exists():
        print(f"❌ Missing: {compose_file}")
    if not settings_file.exists():
        print(f"❌ Missing: {settings_file}")
    
    # Check Docker status
    docker_result = run_command(["docker", "info"], check=False)
    if docker_result.returncode != 0:
        print("❌ Docker daemon is not running")
    
    # Check if port is already in use
    print("\n🔧 If port 8080 is already in use by another SearxNG instance:")
    print("   1. Stop existing containers: docker stop $(docker ps -q)")
    print("   2. Or modify the port in config/searxng/docker-compose.yml")
    
    print("\nPlease check the error messages above and try running the cell again")

## Step 6: Verify SearxNG Service

In [ ]:
import requests
import json

def test_searxng():
    """Test if SearxNG is responding"""
    try:
        response = requests.get("http://localhost:8080/search", 
                              params={"q": "test", "format": "json"}, 
                              timeout=10)
        if response.status_code == 200:
            data = response.json()
            results_count = len(data.get("results", []))
            print(f"✅ SearxNG is working! Found {results_count} test results")
            return True
        else:
            print(f"⚠️  SearxNG responded with status {response.status_code}")
            return False
    except requests.exceptions.RequestException as e:
        print(f"❌ Cannot connect to SearxNG: {e}")
        return False

# Test SearxNG
print("🔍 Testing SearxNG service...")
searxng_working = test_searxng()

if not searxng_working:
    print("\n🔧 Troubleshooting steps:")
    print("1. Check if Docker containers are running:")
    print("   docker ps")
    print("2. Check Docker logs:")
    print("   docker logs <container_id>")
    print("3. Try restarting the setup:")
    print("   bash scripts/setup_standalone.sh")

## Step 7: Launch Deep Search Testing Notebook

In [ ]:
# Install Jupyter if not present in the environment
print("📓 Setting up Jupyter notebook environment...")

try:
    # Check if jupyter is available in the UV environment
    result = run_command(["uv", "run", "jupyter", "--version"], check=False)
    if result.returncode != 0:
        print("📦 Installing Jupyter...")
        run_command(["uv", "add", "jupyter"])
    
    print("✅ Jupyter is ready")
    
    # Check if the deep search testing notebook exists
    notebook_path = Path("notebooks/deep_search_testing.ipynb")
    if notebook_path.exists():
        print(f"✅ Found notebook: {notebook_path}")
        
        # Display quick instructions
        print("\n🚀 SETUP COMPLETE! 🚀")
        print("\n" + "="*60)
        print("NEXT STEPS:")
        print("="*60)
        print(f"1. The environment is ready at: {workspace_dir}")
        print(f"2. SearxNG search service is running at: http://localhost:8080")
        print(f"3. Open the deep search testing notebook:")
        print(f"   {notebook_path.absolute()}")
        print("\n📝 To run the notebook:")
        print(f"   cd {workspace_dir}")
        print("   uv run jupyter notebook notebooks/deep_search_testing.ipynb")
        print("\n⚠️  IMPORTANT: Add your API keys to .env file for full functionality")
        print("\n✅ You can now proceed with the workshop!")
    else:
        print(f"⚠️  Notebook not found at: {notebook_path}")
        print("Available notebooks:")
        notebook_dir = Path("notebooks")
        if notebook_dir.exists():
            for nb in notebook_dir.glob("*.ipynb"):
                print(f"   - {nb}")
        else:
            print("   No notebooks directory found")
            
except Exception as e:
    print(f"❌ Error setting up Jupyter: {e}")
    print("You can still run the framework using Python scripts")

## Final Verification and Summary

In [ ]:
# Final system check
print("🔍 FINAL SYSTEM VERIFICATION")
print("="*50)

# Check all components
checks = {
    "Docker": check_command_exists("docker"),
    "UV": check_command_exists("uv"),
    "Git": check_command_exists("git"),
    "Repository": Path(".git").exists(),
    "Virtual Environment": Path(".venv").exists(),
    "Environment File": Path(".env").exists(),
    "AKD Package": True  # We'll test this
}

# Test AKD package import
try:
    result = run_command(["uv", "run", "python", "-c", "import akd; print('OK')"], check=False)
    checks["AKD Package"] = "OK" in result.stdout
except:
    checks["AKD Package"] = False

# Test SearxNG
checks["SearxNG Service"] = searxng_working if 'searxng_working' in locals() else False

# Display results
for component, status in checks.items():
    icon = "✅" if status else "❌"
    print(f"{icon} {component}: {'OK' if status else 'FAILED'}")

# Overall status
all_critical_ok = all([
    checks["Docker"], 
    checks["Repository"], 
    checks["Virtual Environment"], 
    checks["AKD Package"]
])

print("\n" + "="*50)
if all_critical_ok:
    print("🎉 WORKSHOP SETUP SUCCESSFUL! 🎉")
    print("\nYou're ready to start the workshop!")
    if not checks["SearxNG Service"]:
        print("\n⚠️  Note: SearxNG service may need to be restarted")
        print("Run: bash scripts/setup_standalone.sh")
else:
    print("❌ SETUP INCOMPLETE")
    print("\nPlease review the failed components above")
    print("Some manual intervention may be required")

print("\n📍 Workshop Location:", workspace_dir)
print("📖 Main Notebook: notebooks/deep_search_testing.ipynb")
print("🔧 API Keys: Add to .env file for full functionality")